In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.models import model_from_json
from tensorflow.keras.models import load_model

In [ ]:
data = pd.read_json("train.json")
data.head()

In [ ]:
# Create list of unique cuisine types
cuisine_list = data['cuisine']
cuisine_compilation = []
for cuisine in cuisine_list:
    cuisine_compilation.append(cuisine)
      
cuis_unique = list(set(cuisine_compilation))
cuis_unique

In [ ]:
# Onehotencoding for cuisine types
label_encoder_cuis = LabelEncoder()
label_encoder_cuis.fit(cuis_unique)
encoded_cuis = label_encoder_cuis.transform(data['cuisine'])
one_hot_cuis = to_categorical(encoded_cuis)

In [ ]:
# Check first encoded item
one_hot_cuis[0]

In [ ]:
# Add one hot cuisine to dataframe
data["one_hot_cuisine"] = list(one_hot_cuis)

In [ ]:
data.head()

In [ ]:
# Organize data frame
data = data[["cuisine", "one_hot_cuisine", "ingredients"]]
data.head()

In [ ]:
# Create one hot encoding for ingredients that are in lists!
#     -Create dictionary
ingredients = data.loc[:,'ingredients']

i_map = {}
i_list = []
counter = 0
for lists in ingredients:
    for items in lists:
        if items not in i_map:
            i_list.append(items)
            i_map[items] = counter
            counter = counter + 1

In [ ]:
dict(list(i_map.items())[0:5])

In [ ]:
# Create one hot encoding for ingredients that are in lists!
#     -Create encoded data
ingredients_encodings = []
for lists in ingredients:
    encoding = [0]*len(i_map)
    for items in lists:
        encoding[i_map[items]] = 1
    ingredients_encodings.append(encoding)

In [ ]:
ingredients_encodings[0]

In [ ]:
# Add encoded ingredients to data frame
data["one_hot_ingredients"] = ingredients_encodings

In [ ]:
data.head()

In [ ]:
# Create variables for train test split
one_hot_cuisine = data['one_hot_cuisine']
one_hot_ingredients = data['one_hot_ingredients']

In [ ]:
# Split data
ing_train, ing_test,cuis_train, cuis_test = train_test_split(one_hot_ingredients, one_hot_cuisine, random_state=1)

In [ ]:
ing_train.head()

In [ ]:
# Convert object into numpy array 
cuis_train = np.array(cuis_train.tolist())
cuis_test = np.array(cuis_test.tolist())
ing_train = np.array(ing_train.tolist())
ing_test = np.array(ing_test.tolist())

In [ ]:
ing_train

In [ ]:
# Create model structure
deep_model = Sequential()
deep_model.add(Dense(units=20, activation='relu', input_dim=6714))
deep_model.add(Dense(units=15, activation='relu'))
deep_model.add(Dense(units=10, activation='relu'))
deep_model.add(Dense(units=20, activation='softmax'))

In [ ]:
deep_model.summary()

In [ ]:
# Compile and fit model
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    ing_train,
    cuis_train,
    epochs=10,
    shuffle=True,
    verbose=2
)

In [ ]:
# Test the model.  Get loss and accuracy.
deep_model_loss, deep_model_accuracy = deep_model.evaluate(
    ing_test, cuis_test, verbose=2)
print(
    f"Deep Neural Network - Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

In [ ]:
# Save the model weights
deep_model.save("cuisine_deep_model_trained.h5")

In [ ]:
# Save the model structure
with open('deep_model_architecture.json', 'w') as f:
    f.write(deep_model.to_json())